# 3.Random Chinese Sentence Generator


Writing a programming which could generate random Chinese sentences based on one grammar. 

In [1]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个 
noun =>   女人 |  篮球 | 桌子 | 小猫 
verb => 看着 | 听着 | 看见 
Adj =>   蓝色的 |  好看的 | 小小的 |  年轻的"""

首先我们先想办法把规则提取出来，可以将字符串中每个=>左边的映射到key，右边映射到value。

In [2]:
#输入题目要求的字符串，提取规则返回字典。这里传入的参数还有输入字符串中划分statement和expression、expression之间分隔符。
def map(simple_grammar,stmt_split='=>', or_split='|'):
    rules = dict() # key is the @statement, value is @expression
    for line in simple_grammar.split('\n'):
        if not line: 
            continue   # skip the empty line
        #print(line)
        stmt, expr = line.split(stmt_split)
        expr=expr.replace('null',' ')    #不替换成空，为了下一行split可以分开，空格之后可以处理掉。其实这里null后面本来就有空格
        #print(expr)
        rules[stmt.strip()] = expr.split(or_split)
    return rules

In [3]:
print(map(simple_grammar))

{'sentence': [' noun_phrase verb_phrase'], 'noun_phrase': [' Article Adj* noun'], 'Adj*': ['   ', ' Adj Adj*'], 'verb_phrase': [' verb noun_phrase'], 'Article': ['  一个 ', ' 这个 '], 'noun': ['   女人 ', '  篮球 ', ' 桌子 ', ' 小猫 '], 'verb': [' 看着 ', ' 听着 ', ' 看见 '], 'Adj': ['   蓝色的 ', '  好看的 ', ' 小小的 ', '  年轻的']}


这里可以看到rules里有很多空格，我们会在下一步中去除这些空格。因为去除空格需要遍历列表中每一个字符串，而下一步的确也会这么做所以届时去除就可以了。

下一步就是根据提取出的rules，生成句子。可以看到Adj* 中也包含了 Adj* ，所以这个生成显然是递归完成的。

#### 递归边界可以设计为当输入不再是statement而是需要生成的字符串。(target not in rules,即target是'一个'，'小猫'这类而不是'noun_phrase','verb_phrase'这类)

In [4]:
import random
rules=map(simple_grammar)
global i,j                             #添加调用计数，追踪递归过程
i=1
j=1
def generate(rules,target='sentence',trace=1):#trace代表是否追踪生成过程
    global i,j
    if target in rules:                       #rules.key=statement，target是'noun_phrase'这类时
        candidates = rules[target]            #candidates=rules['noun_phrase']=[' Article Adj* noun']
        candidate = random.choice(candidates) #candidate还是[' Article Adj* noun']，因为这个列表就这一个元素
        if trace==1:
            print('调用函数第%d次，target=%s，candidate=%s'%(i,target,candidate))
        i+=1        
        return ''.join(generate(rules, target=c.strip(),trace=trace) for c in candidate.split())  #candidate.split分出c=Articl。c.strip()把每个expression中的空格去除
    else:                                 #target是'小猫'这类时，为递归边界直接返回target
        if trace==1:
            print('调用函数第%d次，target=%s，达到递归边界得到生成的第%d个词---------------------我是分割线----------------------'%(i,target,j))
        i+=1
        j+=1
        return target

In [5]:
generate(rules, target='sentence',trace=0)

'一个年轻的好看的女人看见这个女人'

In [6]:
#封装一下
def solution(simple_grammar,target='sentence',stmt_split='=>', or_split='|',trace=1):
    rules=map(simple_grammar,stmt_split=stmt_split, or_split=or_split)              #提取规则
    global i,j                             
    i=1
    j=1
    o=generate(rules, target=target,trace=trace)   #递归生成句子
    print(o)

In [7]:
solution(simple_grammar)

调用函数第1次，target=sentence，candidate= noun_phrase verb_phrase
调用函数第2次，target=noun_phrase，candidate= Article Adj* noun
调用函数第3次，target=Article，candidate= 这个 
调用函数第4次，target=这个，达到递归边界得到生成的第1个词---------------------我是分割线----------------------
调用函数第5次，target=Adj*，candidate= Adj Adj*
调用函数第6次，target=Adj，candidate= 小小的 
调用函数第7次，target=小小的，达到递归边界得到生成的第2个词---------------------我是分割线----------------------
调用函数第8次，target=Adj*，candidate= Adj Adj*
调用函数第9次，target=Adj，candidate= 小小的 
调用函数第10次，target=小小的，达到递归边界得到生成的第3个词---------------------我是分割线----------------------
调用函数第11次，target=Adj*，candidate= Adj Adj*
调用函数第12次，target=Adj，candidate=   蓝色的 
调用函数第13次，target=蓝色的，达到递归边界得到生成的第4个词---------------------我是分割线----------------------
调用函数第14次，target=Adj*，candidate=   
调用函数第15次，target=noun，candidate=   女人 
调用函数第16次，target=女人，达到递归边界得到生成的第5个词---------------------我是分割线----------------------
调用函数第17次，target=verb_phrase，candidate= verb noun_phrase
调用函数第18次，target=verb，candidate= 听着 
调用函数第19次，target=听着，达到递归边界得到生成的第6个词---

In [8]:
solution(simple_grammar,trace=0)

一个女人听着这个篮球


In [9]:
solution(simple_grammar)

调用函数第1次，target=sentence，candidate= noun_phrase verb_phrase
调用函数第2次，target=noun_phrase，candidate= Article Adj* noun
调用函数第3次，target=Article，candidate=  一个 
调用函数第4次，target=一个，达到递归边界得到生成的第1个词---------------------我是分割线----------------------
调用函数第5次，target=Adj*，candidate= Adj Adj*
调用函数第6次，target=Adj，candidate=  年轻的
调用函数第7次，target=年轻的，达到递归边界得到生成的第2个词---------------------我是分割线----------------------
调用函数第8次，target=Adj*，candidate= Adj Adj*
调用函数第9次，target=Adj，candidate= 小小的 
调用函数第10次，target=小小的，达到递归边界得到生成的第3个词---------------------我是分割线----------------------
调用函数第11次，target=Adj*，candidate=   
调用函数第12次，target=noun，candidate= 桌子 
调用函数第13次，target=桌子，达到递归边界得到生成的第4个词---------------------我是分割线----------------------
调用函数第14次，target=verb_phrase，candidate= verb noun_phrase
调用函数第15次，target=verb，candidate= 听着 
调用函数第16次，target=听着，达到递归边界得到生成的第5个词---------------------我是分割线----------------------
调用函数第17次，target=noun_phrase，candidate= Article Adj* noun
调用函数第18次，target=Article，candidate=  一个 
调用函数第19次，target=一个，达到

再来一个好玩的生成代码。

In [10]:
simpel_programming = '''
if_stmt => if ( cond ) { stmt }
cond => var op var
op => | == | < | >= | <= 
stmt => assign | if_stmt
assign => var = var
var =>  char var | char
char => a | b |  c | d | 0 | 1 | 2 | 3
'''

In [11]:
for i in range(11):
    solution(simpel_programming, target='if_stmt', stmt_split='=>',trace=0) #trace=0关闭追踪

if(cd3>=c3){if(3101<0){d=1d}}
if(3ac){a=0}
if(22310){3bd=cb}
if(13){2300=c}
if(cb2){2=a2d}
if(d<=2){if(b2==d){b=a}}
if(2a0db){d10c0=22}
if(b02ccb){2=b}
if(3<=bd){2=a}
if(a>=a){000=3}
if(20==103ab){accd030=3}
